In [1]:
import wrds
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Read stocks
stock = pd.read_csv('stock1.csv', index_col=0)
s2 = pd.read_csv('stock2.csv', usecols=['ret']).squeeze()
s3 = pd.read_csv('stock3.csv', usecols=['ret']).squeeze()
s4 = pd.read_csv('stock4.csv', usecols=['ret']).squeeze()
s5 = pd.read_csv('stock5.csv', usecols=['ret']).squeeze()
s6 = pd.read_csv('stock6.csv', usecols=['ret']).squeeze()
s7 = pd.read_csv('stock7.csv', usecols=['ret']).squeeze()
s8 = pd.read_csv('stock8.csv', usecols=['ret']).squeeze()
s9 = pd.read_csv('stock9.csv', usecols=['ret']).squeeze()
s10 = pd.read_csv('stock10.csv', usecols=['ret']).squeeze()
s11 = pd.read_csv('stock11.csv', usecols=['ret']).squeeze()
s12 = pd.read_csv('stock12.csv', usecols=['ret']).squeeze()
s13 = pd.read_csv('stock13.csv', usecols=['ret']).squeeze()
s14 = pd.read_csv('stock14.csv', usecols=['ret']).squeeze()
s15 = pd.read_csv('stock15.csv', usecols=['ret']).squeeze()
s16 = pd.read_csv('stock16.csv', usecols=['ret']).squeeze()
s17 = pd.read_csv('stock17.csv', usecols=['ret']).squeeze()
s18 = pd.read_csv('stock18.csv', usecols=['ret']).squeeze()
s19 = pd.read_csv('stock19.csv', usecols=['ret']).squeeze()
s20 = pd.read_csv('stock20.csv', usecols=['ret']).squeeze()

stock.date = pd.to_datetime(stock.date)
ret = pd.DataFrame({'date': stock.date, 's1': stock.ret, 's2':s2,'s3':s3, 's4':s4, 's5':s5, 's6':s6, 's7':s7,
                   's8':s8, 's9':s9, 's10':s10, 's11':s11, 's12':s12, 's13':s13, 's14':s14, 's15':s15,
                   's16':s16, 's17':s17, 's18':s18, 's19':s19, 's20':s20 })
ret = ret.set_index('date')

In [3]:
rf = pd.read_csv('rf.csv', usecols=['tmytm']).squeeze()
#Compute the monthly simple rate
rf_m = (np.exp(rf/100)-1)/12

#### a)

Functions for different portfolio weights

In [ ]:
def w_tan(cov, mu, rf):
    cov_inv = np.linalg.inv(cov)
    A = cov_inv.sum()
    B = cov_inv.sum(0) @ mu
    w = (cov_inv @ (mu-rf))/(B-A*rf)
    return w

def w_min(cov):
    cov_inv = np.linalg.inv(cov)
    return cov_inv.sum(1)/cov_inv.sum()

def w_rp(rets):
    sd = np.apply_along_axis(np.std, 0, rets)
    sd_inv = 1/sd
    return sd_inv/sd_inv.sum()

def w_ew(rets):
    n = rets.shape[1]
    return np.full(n,1/n)

Loop set-up, create portfolio monthly returns array

In [123]:
trade_months = (2020-1988+1)*12
trade_periods = range(120,ret.shape[0])
n = ret.shape[1]
pfs_rets = np.empty((trade_months,4))

Compute monthly rebalanced returns with 10-year rolling window

In [124]:
for i in trade_periods:
    train = ret.iloc[(i-120):i,:]
    rets = np.array(ret.iloc[i,:])
    cov = np.cov(train, rowvar=False)
    mu = np.apply_along_axis(np.mean, 0, train)
    rff = rf_m[i-120]
    
    wtan = w_tan(cov,mu,rff)
    wmin = w_min(cov)
    wrp = w_rp(train)
    wew = w_ew(train)
    
    W = np.stack((wtan,wmin,wrp,wew), axis=1)
    pfs_ret = rets @ W
    pfs_rets[i-120,:] = pfs_ret
       

In [157]:
pf_dates = ret.index
pf_dates = pf_dates[120:]
pfs_retss = pd.DataFrame(pfs_rets)
pfs_retss['date'] = pf_dates
pfs_retss.set_index('date')
pfs_retss.rename({0:'TAN',1:'MIN',2:'RP',3:'EW'})
pfs_retss


,0,1,2,3,date
0,-0.160205,0.018610,0.037312,0.041143,1988-01-29
1,0.093124,0.011767,0.044241,0.048118,1988-02-29
2,0.081497,-0.057729,-0.034916,-0.032432,1988-03-31
3,0.047814,0.038754,0.016548,0.016976,1988-04-29
4,0.081039,0.018833,0.021081,0.023103,1988-05-31
...,...,...,...,...,...
391,0.151606,0.075923,0.052117,0.050741,2020-08-31
392,0.068692,0.021074,-0.027456,-0.030192,2020-09-30
393,-0.057397,-0.049000,-0.035461,-0.036529,2020-10-30
394,0.039990,0.047664,0.138716,0.154692,2020-11-30
